In [1]:
#imports
import nibabel as nib
import numpy as np
from scipy import ndimage
from ipywidgets import interact, FloatSlider, widgets
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

In [2]:
#load groundtruth image
img = nib.load("UpsampledSegments/newC4HalfGroundTruth.nii.gz")
volume = img.get_fdata()

In [3]:
#invert the image and find the center piece as ROI
inv_volume = np.array(volume == 0)
ROI_3D = np.zeros(np.shape(volume))
for slice_idx in range(86, 121):
    labels, _ = ndimage.label(inv_volume[:, :, slice_idx])
    center_piece = (labels == labels[200][200])
    
    eroded = ndimage.binary_erosion(center_piece, iterations=4)
    ROI_3D[:, :, slice_idx] = eroded
ROI_3D.shape

(454, 443, 239)

In [4]:
#erode the ROI to avoid bony structure due to upsample error
structure = np.ones((3,3))  # neighborhood
eroded = ndimage.binary_erosion(ROI_3D, iterations=4)

In [5]:
#plot ROI within grouth truth
@widgets.interact(slice_idx=(86, 121))
def view_slice(slice_idx=120):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(eroded[:, :, slice_idx] + volume[:, :, slice_idx], cmap='gray')
    plt.title('ROI of slice' + str(slice_idx))
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(np.where(eroded[:, :, slice_idx], volume[:, :, slice_idx], -2), cmap='gray')
    plt.title('ROI of slice' + str(slice_idx))
    plt.axis('off')

interactive(children=(IntSlider(value=120, description='slice_idx', max=121, min=86), Output()), _dom_classes=…

In [6]:
#load the worst upsampled case to check no bony voxel include
look = nib.load("UpsampledSegments/C4NNUpsampled0.5mm.nii.gz")
v = look.get_fdata()
print(np.sum((ROI_3D + v)>1))
ROI_3D [ROI_3D == v] = 0 #remove overlap manually (about 35 voxels)
ROI_3D = np.int8(ROI_3D)

@widgets.interact(slice_idx=(86, 121))
def view_slice(slice_idx=120):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(ROI_3D[:, :, slice_idx] + v[:, :, slice_idx], cmap='gray')
    plt.title('ROI of slice' + str(slice_idx))
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(np.where(ROI_3D[:, :, slice_idx], v[:, :, slice_idx], -2), cmap='gray')
    plt.title('ROI of slice' + str(slice_idx))
    plt.axis('off')

35


interactive(children=(IntSlider(value=120, description='slice_idx', max=121, min=86), Output()), _dom_classes=…

In [7]:
final_img = nib.Nifti1Image(ROI_3D, img.affine)
nib.save(final_img, r"erode_C4ROI3D.nii.gz")

In [8]:
#the .npy file used in simulation is too big to upload to github
#np.save("erode_C4ROI3D.npy", ROI_3D)